# libraries, helper functions and constants

Here are all the libraries that we used for question 16 and some untility functions such as get_cluster_metrics which output the metrics.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from umap import UMAP
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from hdbscan import HDBSCAN
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.cluster import (
    contingency_matrix,
    homogeneity_score,
    v_measure_score,
    completeness_score,
    adjusted_rand_score,
    adjusted_mutual_info_score
)
from plotmat import plot_mat

def get_cluster_metrics(y_true, y_pred, metrics=None):
    if not metrics:
        metrics = [
            homogeneity_score,
            completeness_score,
            v_measure_score,
            adjusted_rand_score,
            adjusted_mutual_info_score
        ]
    d = {}
    for m in metrics:
        d[m.__name__] = m(y_true, y_pred)
    df = pd.DataFrame(d, index=[0]).T
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'metric', 0: 'score'}, inplace=True)
    return df

RANDOM_SEED = 42

# load in data and transform to TF-IDF vectors

We load the data into a pandas dataframe from csv file, and uses factorize function to assign unique ID to each category. category_id_df dataframe holds category name and corresponding ID, and labels dataframe holds the label corresponding to training data.

In [2]:
df = pd.read_csv("./BBC_News_Train.csv")
df['category_id'], _ = df['Category'].factorize()
category_id_df = df[['Category', 'category_id']].drop_duplicates()
labels = df.category_id

Here we uses Term Frequency-Inverse Document Frequency (TF-IDF) metric to build representation vectors for training.

In [3]:
tfidf = TfidfVectorizer(min_df=5, stop_words='english')
tfidf_data = tfidf.fit_transform(df.Text)

# dimension reduction (Truncated SVD, NMF and UMAP with K-mean)

We evaluated three different dimension reduction method (Truncated SVD, NMF and UMAP) with components number ranging from 1 to 800. We evaluate the effectiveness of the dimension reduction using K-mean and clustering algorithm. The performance resulting from each dimension reduction data are shown in tables below. 

In [4]:
km = KMeans(
    n_clusters=category_id_df.Category.count(),
    random_state=0,
    max_iter=5000,
    n_init=50
)

n_components = [1, 2, 3, 5, 10, 20, 50, 100, 300, 500, 800, 1000]

## Truncated SVD

In [5]:
scores = []
for r in n_components:
    input_svd = TruncatedSVD(
        n_components=r, random_state=RANDOM_SEED).fit_transform(tfidf_data)
    preds = km.fit_predict(input_svd)
    scores.append(get_cluster_metrics(labels, preds)['score'].tolist())
metrics = get_cluster_metrics(labels, preds)['metric'].tolist()
svd_metrics_df = pd.DataFrame(scores, columns=metrics, index=n_components)
svd_metrics_df

,homogeneity_score,completeness_score,v_measure_score,adjusted_rand_score,adjusted_mutual_info_score
1,0.094331,0.104362,0.099094,0.049817,0.095885
2,0.303752,0.346610,0.323769,0.193236,0.321323
3,0.448852,0.502422,0.474129,0.355048,0.472247
5,0.636503,0.685090,0.659903,0.572929,0.658711
10,0.633928,0.707318,0.668616,0.543747,0.667433
20,0.616071,0.693878,0.652664,0.514318,0.651419
50,0.635522,0.718284,0.674373,0.536805,0.673204
100,0.620452,0.698453,0.657146,0.517965,0.655918
300,0.715419,0.757440,0.735830,0.662944,0.734912
500,0.782276,0.799019,0.790559,0.780867,0.789844


## NMF

In [6]:
scores = []
for r in n_components:
    inputs_nmf = NMF(n_components=r, init='random',
                     random_state=RANDOM_SEED).fit_transform(tfidf_data)
    preds = km.fit_predict(inputs_nmf)
    scores.append(get_cluster_metrics(labels, preds)['score'].tolist())
metrics = get_cluster_metrics(labels, preds)['metric'].tolist()
nmf_metrics_df = pd.DataFrame(scores, columns=metrics, index=n_components)
nmf_metrics_df

/home/boyuan/.local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/home/boyuan/.local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
/home/boyuan/.local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


,homogeneity_score,completeness_score,v_measure_score,adjusted_rand_score,adjusted_mutual_info_score
1,0.094331,0.104362,0.099094,0.049817,0.095885
2,0.247705,0.308859,0.274922,0.127401,0.272187
3,0.477529,0.493823,0.485539,0.343355,0.483772
5,0.663295,0.708868,0.685325,0.612784,0.684226
10,0.472978,0.627064,0.539229,0.319264,0.537447
20,0.254254,0.456657,0.326642,0.074724,0.323695
50,0.143036,0.427838,0.214395,0.049344,0.210228
100,0.068191,0.370604,0.115188,0.011561,0.109439
300,0.084514,0.349946,0.136147,0.015954,0.131116
500,0.032743,0.290324,0.058849,0.005426,0.052556


## UMAP

For UMAP, we tested on both Eucledian and Cosine distance, and the result are shown below 

### euclidean

In [7]:
scores = []
for n in n_components:
    umapfit = UMAP(n_components = n, metric = 'euclidean')
    inputs_umap = umapfit.fit_transform(tfidf_data)
    preds = km.fit_predict(inputs_umap)
    get_cluster_metrics(labels, preds)
    scores.append(get_cluster_metrics(labels, preds)['score'].tolist())
metrics = get_cluster_metrics(labels, preds)['metric'].tolist()
metrics_df = pd.DataFrame(scores, columns=metrics, index=n_components)
metrics_df

,homogeneity_score,completeness_score,v_measure_score,adjusted_rand_score,adjusted_mutual_info_score
1,0.648914,0.652242,0.650574,0.682402,0.649391
2,0.774596,0.774988,0.774792,0.807148,0.774031
3,0.772620,0.773033,0.772827,0.806810,0.772059
5,0.774802,0.775294,0.775048,0.809163,0.774288
10,0.768688,0.769228,0.768958,0.801502,0.768178
20,0.764969,0.765726,0.765348,0.797540,0.764555
50,0.761241,0.762329,0.761785,0.795600,0.760980
100,0.767745,0.768666,0.768206,0.800770,0.767422
300,0.763184,0.764151,0.763667,0.798226,0.762869
500,0.765523,0.766331,0.765927,0.800668,0.765136


### cosine

In [8]:
scores = []
for n in n_components:
    umapfit = UMAP(n_components = n, metric = 'cosine')
    inputs_umap = umapfit.fit_transform(tfidf_data)
    preds = km.fit_predict(inputs_umap)
    get_cluster_metrics(labels, preds)
    scores.append(get_cluster_metrics(labels, preds)['score'].tolist())
metrics = get_cluster_metrics(labels, preds)['metric'].tolist()
metrics_df = pd.DataFrame(scores, columns=metrics, index=n_components)
metrics_df

,homogeneity_score,completeness_score,v_measure_score,adjusted_rand_score,adjusted_mutual_info_score
1,0.684861,0.683499,0.684179,0.693706,0.683114
2,0.697573,0.695054,0.696311,0.702069,0.695288
3,0.760475,0.762211,0.761342,0.794154,0.760535
5,0.763035,0.763932,0.763483,0.797935,0.762684
10,0.766929,0.767975,0.767451,0.802639,0.766666
20,0.757243,0.758450,0.757846,0.791636,0.757028
50,0.759846,0.760991,0.760418,0.794313,0.759609
100,0.763592,0.764522,0.764057,0.799254,0.763260
300,0.764780,0.765805,0.765292,0.799528,0.764499
500,0.770041,0.770846,0.770443,0.804797,0.769668


As we can see, Truncated SVD achieves the best score of 0.78 to 0.80 in all scoring categories; UMAP score is very close to that of Truncated SVD, and even surpass it in adjusted Rand score; NMF perform the worst, which might be caused by the non-negative weighing constrain making it fail to capture matrix.

# clustering (K-mean, Agglomerative, DBSCAN, HDBSCAN)

Here we use TruncatedSVD with 500 component for K-mean and cosine distance UMAP with 1000 component for Agglomerative, DBSCAN and HDBSCAN (these three work well with UMAP data). The results are shown in the tables below.

In [9]:
svd_train = TruncatedSVD(
    n_components=500, random_state=RANDOM_SEED).fit_transform(tfidf_data)

umap_train = UMAP(
    n_components = 1000, metric = 'cosine').fit_transform(tfidf_data)

## K-mean

In [10]:
preds = km.fit_predict(svd_train)

get_cluster_metrics(labels, preds)

,metric,score
0,homogeneity_score,0.782276
1,completeness_score,0.799019
2,v_measure_score,0.790559
3,adjusted_rand_score,0.780867
4,adjusted_mutual_info_score,0.789844


## Agglomerative

In [11]:
preds = AgglomerativeClustering(n_clusters = category_id_df.Category.count(), linkage = 'ward').fit_predict(umap_train)

get_cluster_metrics(labels, preds)

,metric,score
0,homogeneity_score,0.775997
1,completeness_score,0.776436
2,v_measure_score,0.776217
3,adjusted_rand_score,0.813692
4,adjusted_mutual_info_score,0.775461


## DBSCAN

For DBSCAN, we tested Epsilon ranging from 1 to 21 and min sample ranging from 5 to 200

In [12]:
eps_list = [x * 0.05 for x in range(1, 21)]
min_samples_list = list(range(5,200,10))

scores = []
for ep in eps_list:
    for min_samp in min_samples_list:
        preds = DBSCAN(eps = ep, min_samples = min_samp, n_jobs = -1).fit_predict(umap_train)
        row = get_cluster_metrics(labels, preds)['score'].tolist()
        row.append(ep)
        row.append(min_samp)
        scores.append(row)
        
titles = get_cluster_metrics(labels, preds)['metric'].tolist()
titles.append("Epsilon")
titles.append("min_samples")
metrics_df = pd.DataFrame(scores, columns=titles)
metrics_df.to_excel('DBSCAN.xlsx')

Detailed result can be found in DBSCAN.xlsx, here I picked the one with highest score and show it below

In [13]:
preds = DBSCAN(eps = 0.95, min_samples = 75, n_jobs = -1).fit_predict(umap_train)

get_cluster_metrics(labels, preds)

,metric,score
0,homogeneity_score,0.722026
1,completeness_score,0.647094
2,v_measure_score,0.682510
3,adjusted_rand_score,0.669950
4,adjusted_mutual_info_score,0.681241


## HDBSCAN

For HDBSCAN, we set min cluster size to 100 and tested min cluster size ranging from 1 to 21 and min sample ranging from 5 to 500

In [14]:
clust_eps_list = [x * 0.05 for x in range(1, 21)]
min_samples_list = list(range(5,500,10))

scores = []
for ep in clust_eps_list:
    for min_samp in min_samples_list:
        preds = HDBSCAN(min_cluster_size=100, min_samples = min_samp, cluster_selection_epsilon = ep, core_dist_n_jobs=-1).fit_predict(umap_train)
        row = get_cluster_metrics(labels, preds)['score'].tolist()
        row.append(ep)
        row.append(min_samp)
        scores.append(row)
        
titles = get_cluster_metrics(labels, preds)['metric'].tolist()
titles.append("Cluster_sel_Epsilon")
titles.append("min_samples")
metrics_df = pd.DataFrame(scores, columns=titles)
metrics_df.to_excel('HDBSCAN.xlsx')

Detailed result can be found in HDBSCAN.xlsx, here I picked the one with highest score and show it below

In [15]:
preds = HDBSCAN(min_cluster_size=100, min_samples = 15, cluster_selection_epsilon = 0.05, core_dist_n_jobs=-1).fit_predict(umap_train)

get_cluster_metrics(labels, preds)

,metric,score
0,homogeneity_score,0.554131
1,completeness_score,0.773971
2,v_measure_score,0.645856
3,adjusted_rand_score,0.565880
4,adjusted_mutual_info_score,0.644796


As we can see, K-mean is the best performing algorithm. It outperforms DBSCAN/HDBSCAN and slightly outperforms Agglomerative by a very slim margin. We believe this is caused by the sparse nature of the dataset, which favors K-mean over DBSCAN/HDBSCAN. Moreover, K-mean and Agglomerative have the advantage of knowing the exact number of clusters, while DBSCAN/HDBSCAN doesn't.